# Capstone Project for Coursera
This notebook will be mainly used for capstone project of Coursera online certificate for **IBM Data Science Professional Ceritificate**.

In [97]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Import Libraries

First import the libraries which will be used in this capstone project.

In [98]:
import json

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Libraries imported.


### Import Data

Next, using **BeautifulSoup** to transform the data in the table on the Wikipedia page into pandas dataframe. At the same time, cleanning dataframe as the requirement stated.

In [99]:
from bs4 import BeautifulSoup

with open('List of postal codes of Canada_ M - Wikipedia.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')


In [174]:
t = soup.find('table', class_='wikitable sortable')
table_rows = table.find_all("tr")
table_columns = table.find_all("th")
l = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    if row and row[1] != 'Not assigned':
        if row[2] == 'Not assigned\n':
            row[2] = row[1]
        l.append(row)
cols = [c.text.replace('\n', '') for c in table_columns]
df = pd.DataFrame(l, columns=cols).replace('\n', '', regex=True)
df = df.groupby(['Borough', 'Postcode'], sort=False).agg(lambda x: ', '.join(set(x.dropna()))).reset_index()
df = df[cols].replace('\n', '', regex=True)
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [163]:
df.shape

(103, 3)

As the result of **shape** method, the size of this transformed table is **(103, 3)**.

Next step is add the **Latitude** and **Longitude** to the dataframe.

In [175]:
ll = pd.read_csv('Geospatial_Coordinates.csv')
ll.columns = ['Postcode', 'Latitude', 'Longitude']
cols = ['Postcode']
df_withll = df.join(ll.set_index(cols), on=cols)

### Use geopy library to get the latitude and longitude values of Toronto

In [184]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Ontario with neighborhoods superimposed on top

In [185]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_withll['Latitude'], df_withll['Longitude'], df_withll['Borough'], df_withll['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Then filtering the **Borough** by contains **Toronto**

In [173]:
df_tor = df_withll[df_withll['Borough'].str.contains("Toronto")]
df_tor

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"King, Adelaide, Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
36,M5J,Downtown Toronto,"Toronto Islands, Harbourfront East, Union Station",43.640816,-79.381752


Creating map of Toronto using only Borough that contains Toronto.

In [191]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [192]:
CLIENT_ID = 'M5NXRZI5KMSVEMTB1FHXAKUUXF5ODQFPRDMJQEZ053TRIV4W' # your Foursquare ID
CLIENT_SECRET = 'HBRGKYXZTPLCSDZPS4HQEEGZFOTJ2LKL4N5XMURF0OW3URRB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M5NXRZI5KMSVEMTB1FHXAKUUXF5ODQFPRDMJQEZ053TRIV4W
CLIENT_SECRET:HBRGKYXZTPLCSDZPS4HQEEGZFOTJ2LKL4N5XMURF0OW3URRB
